### Что проверяем:

слова из списка сводеша сохраняют ранги своих соседей независимо от модели

слова из списка сводеша демонстрируют большую устойчивость к набору соседей, чем случайные слова

слова из списка сводеша сохраняют расстояния к частотным словам языка независимо от модели.


In [2]:
# coding: utf-8

'''
This module is formatting  user config with facts and entities,
creating entity index and producing all variants for entity-match
'''

import gensim

import re
import os
from tqdm import tqdm
from operator import itemgetter
from scipy import stats
import random
from functools import lru_cache
import zipfile
from ufal.udpipe import Model, Pipeline

In [73]:
WDIR = r'/media/mi_air/0F0B7DDE62EEA81E/vector/'
modelfile = WDIR+r'english-ud-2.0-170801.udpipe'
modelm = Model.load(modelfile)
process_pipeline = Pipeline(modelm, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

In [74]:
def tag_ud(text):
    for line in text:
        # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
        output = process_pipeline.process(text)
        output = output.split('\n')[4]
    return output.split('\t')[1]+'_'+output.split('\t')[3]


In [75]:
tag_ud('mother')

'mother_NOUN'

In [7]:
def open_model(m):
    try:
        if m.endswith('.vec'):
            model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
        elif m.endswith('.bin'):
            model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)

        else:
            model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False, unicode_errors='replace')
    except:
        model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False, unicode_errors='replace')
    return model

In [72]:
freqwords = open('/media/mi_air/0F0B7DDE62EEA81E/vector/eng_lemma.num', 'r', encoding='utf8').read().split('\n')[1:]
freqwords = [w.split()[2] for w in freqwords if  len(w.split())>2]

In [76]:
#bnc
with zipfile.ZipFile(WDIR + "bnc_en.zip", "r") as archive:
    stream = archive.open("model.txt")
model = open_model(stream)

In [17]:
#bnc
rset1 = list(model.vocab)

In [18]:
#gigaword
with zipfile.ZipFile(WDIR + "gigaword_en.zip", "r") as archive:
    stream = archive.open("model.txt")
model = open_model(stream)

rset2 = list(model.vocab)

In [19]:
#wiki
with zipfile.ZipFile(WDIR + "wiki_en.zip", "r") as archive:
    stream = archive.open("model.txt")
model = open_model(stream)
rset3 = list(model.vocab)

In [20]:
intersec_words = set.intersection(set(rset1),set(rset2),set(rset3))

In [42]:
len(intersec_words)
f = open(WDIR+'intersec_words_english.txt', 'w', encoding='utf8')
f.write('\n'.join(intersec_words))
f.close()

In [19]:
eng_svodesh = [tag_ud(w) for w in eng_svodesh if tag_ud(w) in intersec_words ]

In [20]:
len(eng_svodesh)

160

In [77]:
freq_words = random.sample([tag_ud(w) for w in freqwords[:2000] if tag_ud(w) in intersec_words], len(eng_svodesh))

In [21]:
random_words = random.sample(intersec_words, len(eng_svodesh))

# эксперимент 1 - получаем 100 ближайших соседей по каждой модели для каждого слова

по аранеа
по новостям
по нкря
по вики

затем для каждого слова считаем % пересечения соседей по моделям

In [80]:
dic_svodesh = { 'wiki':{},'gigaword':{}, 'bnc':{}}
dic_random = {'wiki':{},'gigaword':{}, 'bnc':{}}
dic_freq = {'wiki':{},'gigaword':{}, 'bnc':{}}

In [81]:
for w in eng_svodesh:
    if w not in dic_svodesh['bnc']:
        dic_svodesh['bnc'][w] = model.most_similar(w,topn=100)

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [82]:
for w in random_words:
    if w not in dic_random['bnc']:
        dic_random['bnc'][w] = model.most_similar(w,topn=100)

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [85]:
for w in freq_words:
    if w not in dic_freq['bnc']:
        dic_freq['bnc'][w] = model.most_similar(w,topn=100)

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [86]:
#gigaword
with zipfile.ZipFile(WDIR + "gigaword_en.zip", "r") as archive:
    stream = archive.open("model.txt")
model = open_model(stream)

In [87]:
for w in eng_svodesh:
    if w not in dic_svodesh['gigaword']:
        try:
            dic_svodesh['gigaword'][w] = model.most_similar(w,topn=100)
        except:
            dic_svodesh['gigaword'][w] = []

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [88]:
for w in random_words:
    if w not in dic_random['gigaword']:
        try:
            dic_random['gigaword'][w] = model.most_similar(w,topn=100)
        except:
            dic_random['gigaword'][w] = []

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [89]:
for w in freq_words:
    if w not in dic_freq['gigaword']:
        dic_freq['gigaword'][w] = model.most_similar(w,topn=100)

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [90]:
#wiki
with zipfile.ZipFile(WDIR + "wiki_en.zip", "r") as archive:
    stream = archive.open("model.txt")
model = open_model(stream)

In [91]:
for w in eng_svodesh:
    if w not in dic_svodesh['wiki']:
        try:
            dic_svodesh['wiki'][w] = model.most_similar(w,topn=100)
        except:
            dic_svodesh['wiki'][w] = []

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [92]:
for w in random_words:
    if w not in dic_random['wiki']:
        try:
            dic_random['wiki'][w] = model.most_similar(w,topn=100)
        except:
            print(w)
            dic_random['wiki'][w] = []

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [93]:
for w in freq_words:
    if w not in dic_freq['wiki']:
        dic_freq['wiki'][w] = model.most_similar(w,topn=100)

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [94]:
import pandas as pd


In [95]:
def count_set(list1, list2, list3):
    set1, set2, set3 = set(list1), set(list2), set(list3)
    d = set.intersection(set1, set2, set3)
    u = set(list(set1)+list(set2)+list(set3))
    return len(d)/len(u)

In [96]:
def get_words(dic):
    return [i[0] for i in dic]

In [97]:
rows_list = []
for word in eng_svodesh:
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'svodesh'
        dict1['% intersection'] = count_set(get_words(dic_svodesh['bnc'][word]),get_words(dic_svodesh['wiki'][word]),get_words(dic_svodesh['gigaword'][word]))
        rows_list.append(dict1)
for word in random_words:
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'random'
        dict1['% intersection'] = count_set(get_words(dic_random['bnc'][word]),get_words(dic_random['wiki'][word]),get_words(dic_random['gigaword'][word]))
        rows_list.append(dict1)
for word in freq_words:
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'freq'
        dict1['% intersection'] = count_set(get_words(dic_freq['bnc'][word]),get_words(dic_freq['wiki'][word]),get_words(dic_freq['gigaword'][word]))
        rows_list.append(dict1)
df = pd.DataFrame(rows_list) 
df.head()

,% intersection,svodesh/random,word
0,0.014652,svodesh,one_NUM
1,0.039216,svodesh,two_NUM
2,0.064257,svodesh,three_NUM
3,0.075000,svodesh,four_NUM
4,0.079167,svodesh,five_NUM


In [40]:
#svodesh
sv = df[df['svodesh/random']=='svodesh']
#random
rn = df[df['svodesh/random']=='random']

In [41]:
#манн-уитни
from scipy.stats import mannwhitneyu

In [42]:
mannwhitneyu(sorted(sv['% intersection']), sorted(rn['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=18873.0, pvalue=0.999999999999925)

In [ ]:
#Получается, что нулевая гипотеза о том, что степень признака в выборках неравна (в первой меньше) неверна
#принимается альтернативная гипотеза, что в случайной выборке слов степень признака меньше

In [98]:
df.to_csv(path_or_buf='/media/mi_air/0F0B7DDE62EEA81E/vector/closest_word_intersection_eng.csv', sep='\t')

In [17]:
eng_svodesh = eng_svodesh.split('\n')
eng_svodesh = [i.strip() for i in eng_svodesh]

In [18]:
len(eng_svodesh)

205

In [16]:
eng_svodesh = '''I 
you
he 
we 
they 
this 
that 
here 
there 
who 
what 
where 
when 
how 
not 
all 
many 
some 
few 
other 
one 
two 
three 
four 
five 
big 
long 
wide 
thick 
heavy 
small 
short 
narrow 
thin 
woman 
man
child 
wife 
husband 
mother 
father 
animal 
fish 
bird 
dog 
louse 
snake 
worm 
tree 
forest 
stick 
fruit 
seed 
leaf 
root 
bark 
flower 
grass 
rope 
skin 
meat 
blood 
bone 
fat
egg 
horn 
tail 
feather 
hair 
head 
ear 
eye 
nose 
mouth 
tooth 
tongue 
fingernail 
foot 
leg 
knee 
hand 
wing 
belly 
guts 
neck 
back 
breast 
heart 
liver 
drink 
eat 
bite 
suck 
spit 
vomit 
blow 
breathe 
laugh 
see 
hear 
know 
think 
smell 
fear 
sleep 
live 
die 
kill 
fight 
hunt 
hit 
cut 
split 
stab 
scratch 
dig 
swim 
fly 
walk 
come 
lie
sit 
stand 
turn
fall 
give 
hold 
squeeze 
rub 
wash 
wipe 
pull 
push 
throw 
tie 
sew 
count 
say 
sing 
play 
float 
flow 
freeze 
swell 
sun 
moon 
star 
water 
rain 
river 
lake 
sea 
salt 
stone 
sand 
dust 
earth 
cloud 
fog 
sky 
wind 
snow 
ice 
smoke 
fire 
ash 
to burn 
road 
mountain 
red 
green 
yellow 
white 
black 
night 
day 
year 
warm 
cold 
full 
new 
old 
good 
bad 
rotten 
dirty 
straight 
round 
sharp
dull
smooth 
wet 
dry 
correct 
near 
far 
right 
left 
at 
in 
with 
and 
if 
because 
name '''